In [2]:
import os
import cv2
import shutil
import deeplabcut
import numpy as np
import pandas as pd
import subprocess
import datetime 
import math
from joblib import Parallel, delayed
import glob

# import argparse
from matplotlib import pyplot as plt
from PIL import Image
import scipy.stats as stats
# from scipy.linalg import fractional_matrix_power

# import plotly.express as px
# from IPython.display import HTML

# from dash import Dash, dcc, html, Input, Output, no_update, callback
# import plotly.graph_objects as go
# from plotly.subplots import make_subplots
import base64
import random

2023-12-04 13:57:31.487433: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-04 13:57:31.611728: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-04 13:57:32.029473: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/yramakrishna/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:/usr/local/cuda/lib64:
2023-12-04 13:57:32.029553: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic l

Loading DLC 2.3.3...


In [3]:
# Extracting File names, titles and date created to extrach info from their DLC processing

project_name = 'DLC'
your_name = 'YR'

os.chdir('/home/yramakrishna/DeepLabCut/conda-environments/Codes/Stats_Analysis_AGC/All_vids')
h5files = []
vid_to_crop = []

today_1 = datetime.date(2023, 11, 14)
# today_1 = datetime.date.today()
today = str(today_1)
# today = "2023-10-29"
chkdate = today_1.strftime("%b%d")

proj_fold = project_name+'-'+your_name+'-'+today
# print(today)
target = os.path.join(os.getcwd(),proj_fold,'videos')
# print(target)
for file in os.listdir(target):
    if file.endswith("filtered.h5"):
        h5files.append(os.path.join(target,file))
for file in os.listdir(target):   
    if file.endswith("labeled.mp4"):
        vid_to_crop.append(os.path.join(target,file))
        
crop_source = []
if len(h5files)==len(vid_to_crop):
    for i in h5files:
        found = ''
        for j in vid_to_crop:
            x = os.path.splitext(i)[0].rstrip('_filtered.h5')
            y = os.path.splitext(j)[0].rstrip('filtered_labeled.mp4')
            if x==y:
                found = j
        k = os.path.basename(i)
        l = 'DLC' + k.split('DLC')[1] + 'DLC' + k.split('DLC')[2].rstrip('_filtered.h5')

        crop_source.append([i,found, l, chkdate])
c_s = np.array(crop_source)

In [4]:
for i in c_s:
    fl_pth, vid_pth, x, date = i

    cap = cv2.VideoCapture(vid_pth)

    tot_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(vid_pth, tot_frame)

/home/yramakrishna/DeepLabCut/conda-environments/Codes/Stats_Analysis_AGC/All_vids/DLC-YR-2023-11-14/videos/combined_vids_GCDLC_resnet50_DLCNov14shuffle1_1030000_filtered_labeled.mp4 1869912
/home/yramakrishna/DeepLabCut/conda-environments/Codes/Stats_Analysis_AGC/All_vids/DLC-YR-2023-11-14/videos/combined_vidsdownsampledDLC_resnet50_DLCNov14shuffle1_1030000_filtered_labeled.mp4 1869919
/home/yramakrishna/DeepLabCut/conda-environments/Codes/Stats_Analysis_AGC/All_vids/DLC-YR-2023-11-14/videos/combined_vidsdownsampled_72_2_5_5_AGC_claheDLC_resnet50_DLCNov14shuffle1_1030000_filtered_labeled.mp4 1869919


In [58]:
# Extracts confidence of each point in each video, grouped under points.

feature_names = ['RightEye_Pupil','LeftEye_Pupil']#, 'NostrilsTop_Centre', 'OutlineTop_Mid']#, 'RightEye_Inner', 'LeftEye_Inner']
# feature_names = ['RightEye_Pupil', 'LeftEye_Pupil', 'OutlineTop_Mid', 'NostrilsTop_Centre', 'RightEye_Outer', 'RightEye_Top', 'RightEye_Bottom', 'RightEye_Inner', 'LeftEye_Outer', 'LeftEye_Top', 'LeftEye_Bottom', 'LeftEye_Inner', 'RightNostrils_Top', 'RightNostrils_Bottom','LeftNostrils_Top', 'LeftNostrils_Bottom']


conf = pd.DataFrame()

for i in feature_names:
    df2 = pd.DataFrame()
    mul_cols = []
    for j in c_s:
        fl_pth, vid_pth, x, date = j
        name_check = os.path.basename(vid_pth).split('DLC')[0]
        if name_check.endswith("GC"):
            col_name = "MGC"
        elif name_check.endswith("downsampled"):
            col_name = "NGC"
        else:
            col_name = "AGC"
        tracking_data = pd.read_hdf(fl_pth)
        heading = 'DLC_resnet50_DLC'+ date +'shuffle1_1030000'
        necessary_points = tracking_data[heading][feature_names]
        
        lik = pd.DataFrame(necessary_points[i].likelihood)
        lik = lik.rename(columns={"likelihood" : col_name})
        df2 = pd.concat([df2, lik], axis = 1)
        mul_cols.append((i, col_name))

    df2.columns=pd.MultiIndex.from_tuples(mul_cols)
    conf = pd.concat([conf, df2], axis = 1)


In [12]:
man_lab = pd.read_csv('/home/yramakrishna/DeepLabCut/conda-environments/Codes/Stats_Analysis_AGC/All_vids/DLC-YR-2023-12-03/labeled-data/Frames_to_label_2023-12-03/CollectedData_YR.csv', header=[1,2])

In [9]:
origin = pd.read_csv('/home/yramakrishna/DeepLabCut/conda-environments/Codes/Stats_Analysis_AGC/All_vids/Frame_source_info.csv')

In [31]:
name = man_lab[('Unnamed: 2_level_0', 'Unnamed: 2_level_1')]

frame_no = []

for i in name:
    frame_no.append(int(i.split('img')[1].split('.png')[0]))

In [56]:
frames = origin.iloc[frame_no]
frames.columns = ['frame','gc']

,frame,gc
83,552,0
129,631,0
187,730,1
623,3005,0
891,8279,0
...,...,...
34162,1859717,1
34549,1862119,1
34701,1862274,1
34849,1867359,1


In [108]:
landmarks = pd.DataFrame()

for i in feature_names:
    df2 = pd.DataFrame()
    mul_cols = []
    done = 0
    for j in c_s:
        fl_pth, vid_pth, x, date = j
        name_check = os.path.basename(vid_pth).split('DLC')[0]
        if name_check.endswith("GC"):
            continue
        elif name_check.endswith("downsampled"):
            col_name = "NGC"
        else:
            col_name = "AGC"
        tracking_data = pd.read_hdf(fl_pth)
        trdt = tracking_data.loc[frames.frame]
        necessary_points = trdt[heading][feature_names]
        x = pd.DataFrame(necessary_points[i])
        x = x.drop(['likelihood'], axis = 1)
        x.reset_index(drop=True, inplace=True)
        # print(x)
        if done == 0:
            ml = pd.DataFrame(man_lab[i])
            df2 = pd.concat([df2, ml], axis = 1, ignore_index=True)
            mul_cols.append((i, 'Manual', ml.columns[0]))
            mul_cols.append((i, 'Manual', ml.columns[1]))
            # print(ml)
            done = 1
        
        df2 = pd.concat([df2, x], axis = 1, ignore_index=True)
        mul_cols.append((i, col_name, x.columns[0]))
        mul_cols.append((i, col_name, x.columns[1]))
    
    # print(df2)
    df2.columns=pd.MultiIndex.from_tuples(mul_cols)
    landmarks = pd.concat([landmarks, df2], axis = 1)

In [109]:
landmarks

RightEye_Pupil                                                            
            Manual                    NGC                    AGC              
                 x          y           x          y           x          y   
0       163.337610  71.798034  168.057480  81.420334  168.771103  80.643082  \
1       166.886349  74.237792  172.617477  85.908699  173.559753  85.447815   
2       174.871011  78.230123  177.536346  88.249443  176.486755  88.208099   
3       162.228629  76.233958  169.079529  88.816162  168.673859  86.927513   
4       202.373736  73.350608  177.661285  81.910545  178.036377  80.058510   
..             ...        ...         ...        ...         ...        ...   
96      195.369332  72.100653  200.387787  86.044708  199.551773  84.788300   
97      165.889543  74.601968  171.739380  84.851639  172.261871  84.748604   
98      163.745558  74.244637  169.014908  85.713341  169.230103  85.540588   
99      185.721401  76.388622  194.154053  90.065163  192.461731  87.898056   
100     163.745558  78.175276  171.012405  90.295364  171.625565  90.512161   

    LeftEye_Pupil                                                             
           Manual                    NGC                     AGC              
                x          y           x           y           x           y  
0      196.163443  74.237792  200.270126   82.670647  199.739624   82.691681  
1      196.828832  79.560900  206.423599   90.909775  206.335175   89.752518  
2      203.926309  87.545563  208.256821   96.062485  208.685089   95.876312  
3      195.054463  78.895512  200.988297   86.858582  200.474808   86.521561  
4      171.100476  69.801869  208.452133   85.366043  208.147614   85.274437  
..            ...        ...         ...         ...         ...         ...  
96     221.454479  94.076495  225.264236  104.603775  224.160172  106.021179  
97     198.585309  81.569918  205.179886   92.578621  204.080841   92.298424  
98     195.190667  82.641911  202.669312   95.378304  201.610443   93.903275  
99     211.270552  93.540499  217.809631  106.668152  216.385101  107.543533  
100    196.262659  86.572549  203.001724   96.548012  202.025146   95.956360  

[101 rows x 12 columns]

In [132]:
comp = ['NGC', 'AGC']

point_diff = pd.DataFrame()
mulcol = []
for i in feature_names:
    root = landmarks[i]['Manual']
    
    for j in comp:
        mulcol.append((i, j))
        check = landmarks[i][j]
        xdiff = (root['x'] - check['x'])**2
        ydiff = (root['y'] - check['y'])**2
        d = (xdiff + ydiff)**(1/2)
        diff = pd.DataFrame(d, columns=[j])
        point_diff = pd.concat([point_diff, diff], axis = 1)

print(point_diff)
print(mulcol)
point_diff.columns = pd.MultiIndex.from_tuples(mulcol)

           NGC        AGC        NGC        AGC
0    10.717547  10.380641   9.379652   9.179178
1    13.002150  13.046032  14.861242  13.936987
2    10.367776  10.107949   9.554648   9.594132
3    14.326433  12.485716   9.930804   9.356108
4    26.152968  25.244861  40.464674  40.148359
..         ...        ...        ...        ...
96   14.819635  13.359236  11.195440  12.247295
97   11.801540  11.981685  12.832770  12.054116
98   12.621300  12.557019  14.769761  12.962710
99   16.067277  13.337883  14.666111  14.907836
100  14.131653  14.638758  12.038474  11.011910

[101 rows x 4 columns]
[('RightEye_Pupil', 'NGC'), ('RightEye_Pupil', 'AGC'), ('LeftEye_Pupil', 'NGC'), ('LeftEye_Pupil', 'AGC')]


In [133]:
point_diff

RightEye_Pupil            LeftEye_Pupil           
               NGC        AGC           NGC        AGC
0        10.717547  10.380641      9.379652   9.179178
1        13.002150  13.046032     14.861242  13.936987
2        10.367776  10.107949      9.554648   9.594132
3        14.326433  12.485716      9.930804   9.356108
4        26.152968  25.244861     40.464674  40.148359
..             ...        ...           ...        ...
96       14.819635  13.359236     11.195440  12.247295
97       11.801540  11.981685     12.832770  12.054116
98       12.621300  12.557019     14.769761  12.962710
99       16.067277  13.337883     14.666111  14.907836
100      14.131653  14.638758     12.038474  11.011910

[101 rows x 4 columns]

In [134]:
for i in feature_names:
    print(i)
    print(stats.ttest_ind(point_diff[i]['NGC'], point_diff[i]['AGC']))

RightEye_Pupil
Ttest_indResult(statistic=0.7930399002387447, pvalue=0.42869434618885827)
LeftEye_Pupil
Ttest_indResult(statistic=nan, pvalue=nan)
